# Explicación del Funcionamiento del Sistema RAG

Este notebook explica de manera simple y paso a paso cómo funciona el sistema completo de Generación Aumentada por Recuperación (RAG) para consultar discursos del Presidente Javier Milei. Cubriremos desde la ingesta inicial de datos hasta la consulta final en la interfaz.

**Requisitos previos:**
- Archivo `mi_corpus.json` con los discursos procesados
- Variable de entorno `GOOGLE_API_KEY` configurada con tu clave de API de Google
- Bibliotecas instaladas (ver requirements.txt)

In [ ]:
# Importaciones necesarias
import sys
import os

# Agregar el directorio padre al path para importar módulos locales
sys.path.append('..')

# Importar funciones del sistema RAG
from mvp_rag import load_discursos_from_file, crear_vectorstore, configurar_rag, consultar_rag

# Verificar que la API key esté configurada
if not os.getenv('GOOGLE_API_KEY'):
    print("⚠️  Advertencia: GOOGLE_API_KEY no está configurada. Configúrala antes de continuar.")
else:
    print("✅ GOOGLE_API_KEY configurada correctamente.")

## 1. Generación del Corpus

El primer paso en el sistema RAG es crear un corpus de documentos. Esto implica:

1. **Scraping de datos:** Recopilar discursos desde fuentes externas (como el sitio web de Casa Rosada)
2. **Procesamiento de textos:** Limpiar y estructurar el contenido de los discursos
3. **Almacenamiento:** Guardar los discursos en un archivo JSON para uso posterior

En este proyecto, el corpus ya está preprocesado y almacenado en `mi_corpus.json`. Cada documento contiene:
- `titulo`: Título del discurso
- `contenido_limpio`: Texto procesado del discurso
- `fecha_publicacion`: Fecha del discurso
- `url`: Enlace al discurso original

**Nota:** Para generar un nuevo corpus desde cero, se ejecutaría `generate_corpus.py`, que utiliza scraping web.

In [ ]:
# Cargar el corpus de discursos desde el archivo
print("Cargando discursos desde mi_corpus.json...")
discursos = load_discursos_from_file()
print(f"✅ Se cargaron {len(discursos)} discursos exitosamente.")

# Mostrar información básica del corpus
print(f"\n📊 Estadísticas del corpus:")
print(f"- Número total de discursos: {len(discursos)}")

# Mostrar un ejemplo de discurso
if discursos:
    ejemplo = discursos[0]
    print(f"\n📄 Ejemplo de discurso:")
    print(f"Título: {ejemplo['titulo']}")
    print(f"Fecha: {ejemplo['fecha_publicacion']}")
    print(f"Contenido (primeros 200 caracteres): {ejemplo['contenido_limpio'][:200]}...")
    print(f"URL: {ejemplo['url']}")
else:
    print("❌ No se encontraron discursos en el archivo.")

## 2. Procesamiento y Creación del Vectorstore

Una vez que tenemos el corpus, necesitamos procesarlo para crear un vectorstore que permita búsquedas eficientes:

1. **División de documentos:** Los discursos largos se dividen en fragmentos más pequeños (chunks) para mejor recuperación
2. **Embeddings:** Cada fragmento se convierte en un vector numérico usando un modelo de embeddings
3. **Vectorstore:** Los vectores se almacenan en una base de datos vectorial (ChromaDB) para búsquedas por similitud

El sistema utiliza diferentes estrategias de división según la longitud del documento:
- Documentos largos (>5000 caracteres): Chunks de 1000 caracteres con 200 de solapamiento
- Documentos cortos (<1000 caracteres): Chunks de 300 caracteres con 50 de solapamiento
- Documentos intermedios: Chunks de 500 caracteres con 50 de solapamiento

Se utiliza el modelo de embeddings `intfloat/multilingual-e5-large` para manejar texto en español.

In [ ]:
# Crear el vectorstore a partir de los discursos cargados
print("Creando vectorstore... Esto puede tomar unos minutos.")
vectorstore = crear_vectorstore(discursos)
print("✅ Vectorstore creado y persistido exitosamente.")

# Verificar el vectorstore
print(f"\n📊 Información del vectorstore:")
print(f"- Nombre de colección: documentos_discursos")
print(f"- Directorio de persistencia: ./chroma_db")

# Probar una búsqueda simple en el vectorstore
print("\n🔍 Probando búsqueda en vectorstore...")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
resultados_prueba = retriever.get_relevant_documents("economía")
print(f"Se encontraron {len(resultados_prueba)} documentos relevantes para 'economía':")
for i, doc in enumerate(resultados_prueba, 1):
    print(f"{i}. {doc.metadata['titulo']} ({doc.metadata['fecha_publicacion']})")

## 3. Configuración del Sistema RAG

Con el vectorstore listo, configuramos el sistema RAG completo:

1. **Modelo de Lenguaje (LLM):** Se utiliza Google Gemini 2.5-flash como modelo generativo
2. **Prompt Template:** Plantilla especializada para consultas sobre discursos de Milei
3. **Retriever:** Configurado con MMR (Maximal Marginal Relevance) para diversidad en resultados
4. **Cadena RAG:** Combinación de recuperación y generación usando RetrievalQA

El prompt está diseñado específicamente para:
- Proporcionar información precisa basada únicamente en discursos oficiales
- Estructurar respuestas con resumen ejecutivo, detalles específicos y fuentes
- Mantener un tono profesional y amigable
- Indicar claramente cuando información no está disponible

In [ ]:
# Configurar el sistema RAG
print("Configurando sistema RAG...")
sistema_rag = configurar_rag(vectorstore)
print("✅ Sistema RAG configurado exitosamente.")

# Mostrar configuración del retriever
print(f"\n⚙️  Configuración del retriever:")
print(f"- Tipo de búsqueda: MMR (Maximal Marginal Relevance)")
print(f"- Número de documentos a recuperar: 5")
print(f"- Documentos candidatos iniciales: 15")
print(f"- Parámetro lambda: 0.6")

# Mostrar configuración del LLM
print(f"\n🤖 Configuración del LLM:")
print(f"- Modelo: gemini-2.5-flash")
print(f"- Temperatura: 0.1")
print(f"- Proveedor: Google Generative AI")

## 4. Consulta en el Sistema RAG

Finalmente, podemos realizar consultas al sistema RAG. El proceso funciona así:

1. **Consulta del usuario:** Se recibe una pregunta en lenguaje natural
2. **Recuperación:** El retriever busca los documentos más relevantes en el vectorstore
3. **Generación:** El LLM genera una respuesta basada en los documentos recuperados
4. **Respuesta estructurada:** Se devuelve una respuesta con resumen, detalles y fuentes

La interfaz web (app.py) permite consultas interactivas, pero aquí realizaremos una consulta de ejemplo directamente.

In [ ]:
# Realizar una consulta de ejemplo al sistema RAG
pregunta_ejemplo = "¿Qué dice Milei sobre la economía argentina?"

print(f"💬 Realizando consulta: '{pregunta_ejemplo}'")
print("Procesando...")

# Ejecutar la consulta
resultado = consultar_rag(sistema_rag, pregunta_ejemplo)

# Mostrar el resultado
print("\n✅ Respuesta del sistema RAG:")
print("=" * 50)
print(resultado['result'])
print("=" * 50)

# Mostrar información sobre las fuentes
fuentes = resultado.get('source_documents', [])
print(f"\n📚 Fuentes consultadas ({len(fuentes)} documentos):")
for i, fuente in enumerate(fuentes, 1):
    metadata = fuente.metadata
    print(f"{i}. {metadata.get('titulo', 'Sin título')} - {metadata.get('fecha_publicacion', 'Sin fecha')}")
    print(f"   URL: {metadata.get('url', 'Sin URL')}")

## Conclusión

Hemos visto cómo funciona el sistema RAG completo paso a paso:

1. **Generación del corpus:** Recopilación y procesamiento de discursos
2. **Vectorstore:** Creación de representaciones vectoriales para búsqueda eficiente
3. **Configuración RAG:** Integración de recuperación y generación
4. **Consulta:** Respuestas precisas basadas en fuentes verificables

Este sistema permite consultas inteligentes sobre discursos oficiales, proporcionando respuestas contextualizadas y con referencias a las fuentes originales.

**Para usar en producción:** Ejecuta `streamlit run app.py` para acceder a la interfaz web interactiva.